In [4]:
import re
import jieba
import jieba.analyse
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [17]:
def CUT(df):
    typ=[]
    for i in df['類型']:
        ch=re.compile("\u4e00-\u9fa5")
        seg_word= "".join(ch.findall(i))
        top=jieba.lcut(seg_word)
        typ.append(top)
        
    nam=[]
    for i in df['中文名稱']:
        ch=re.compile("\u4e00-\u9fa5")
        name = "".join(ch.findall(i))
        nam.append(name)
        
    intro=[]
    for i in df['簡介']:
        top=jieba.analyse.extract_tags(i,topK=2)
        intro.append(top)
        
    data={'type':typ ,'name':nam,'article':intro}
    df1=pd.DataFrame(data)
    
    df1['type'] = df1['type'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    df1['article'] = df1['article'].apply(lambda x :  str(x)[1:-2].replace("'","").replace(",",""))
    df1['key'] = df1['name'].astype(str)+' '+df1['type'].astype(str)+' '+df1['article'].astype(str)
    return df1

In [27]:
def COUNT(df):
    vector=CountVectorizer()
    k=vector.fit_transform(df['key'])
    TFidf=TfidfTransformer()
    tf=TFidf.fit_transform(k)
    word=vector.get_feature_names()
    
    return tf

In [28]:
def KNN(X,Y):
    X_train, X_test, y_train, y_test = train_test_split(X, Y['type'].str[0:2], test_size=0.07659)
    clf=KNeighborsClassifier(n_neighbors=51)
    clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)    
    y_test=y_test.values

    print("精準度：",metrics.accuracy_score(y_test, y_pred))
    

In [29]:
df = pd.read_csv('movie.csv')
df = df.drop(labels=['Unnamed: 0'],axis='columns')
df1 = CUT(df) 
tf = COUNT(df1)
KNN(tf,df1)

精準度： 1.0
